# JEDHA-Projet-4b-Convertion_rate

<div style="text-align:left;">
    <img src="./img/img.png" alt="DSW LOGO" width="250px" style="margin-left: 0px;"/>
</div>


#### Description du défi 🚴🚴
Ce projet fais office de compétition de machine learning similaire à celles sur Kaggle, au cours duquel les performances des modèles sont stockées sur un tableau de classement.
Les participants travaillent avec deux fichiers : `data_train.csv`, contenant des données d'entraînement, et `data_test.csv`, avec des données pour les prédictions.

#### Description de l'entreprise 📇
[www.datascienceweekly.org](http://www.datascienceweekly.org) cherche à comprendre le comportement des utilisateurs sur leur site Web pour prédire les abonnements à la newsletter. La compétition implique de construire un modèle pour prédire les conversions, en utilisant des données de trafic Web open source. La métrique d'évaluation est le score f1.

#### Objectifs 🎯
- **Partie 1 :** EDA, prétraitement et entraînement du modèle de base.
- **Partie 2 :** Améliorer le score f1 du modèle avec du feature engineering.
- **Partie 3 :** Faire des prédictions sur `data_test.csv` et les soumettre au tableau de classement.
- **Partie 4 :** Analyser les paramètres du meilleur modèle et recommander des améliorations pour augmenter le taux de conversion.

#### Livrable 📬
- Figures d'EDA.
- Modèle entraîné pour la prédiction des conversions.
- Soumission au tableau de classement.
- Analyse des paramètres du meilleur modèle avec des recommandations exploitables.

# 0.0 - Load & Explore

##
<div style="margin-left: 50px; font-size: 20px;">
0.1 - Import libraries
</div>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, StackingClassifier,VotingClassifier
from xgboost import XGBClassifier
from sklearn.metrics import (accuracy_score,f1_score, ConfusionMatrixDisplay, RocCurveDisplay, mean_absolute_error)
import matplotlib.pyplot as plt
import warnings
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV,cross_val_score
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime
warnings.filterwarnings("ignore", category=DeprecationWarning)
import os

##
<div style="margin-left: 50px; font-size: 20px;">
0.2 - Read File and Clean
</div>

In [ ]:
def describe_df(df):
    print('number of rows:',len(df))
    print('Display of dataset:')
    display(df.head())
    print('Basic statistics:')
    display(df.describe(include="all"))
    print('Pourcentage of missing values:')
    display(df.isnull().sum()/len(df)*100)

In [ ]:
data = pd.read_csv('conversion_data_train.csv')
describe_df(data)

Nous avons un jeu de donnée de 284 500 lignes avant préprocessing.

`features`
- **country** : Pays d'origine de l'utilisateur.
- **age** : Âge de l'utilisateur.
- **new_user** : Indique si l'utilisateur est un nouvel utilisateur ou non.
- **total_pages_visited** : Nombre total de pages visitées par l'utilisateur.

`Target`
- **converted** : Indique si l'utilisateur a converti (1) ou non (0).

`Statistiques des caractéristiques :`
  - **country** : 4 pays uniques, US étant le plus fréquent.
  - **age** : Moyenne d'âge de 30.56 ans, avec un écart type de 8.27 ans.
  - **new_user** : 68.55% des utilisateurs sont de nouveaux utilisateurs.
  - **source** : SEO, ADS, Direct. Seo étant la plus fréquente.
  - **total_pages_visited** : Moyenne de 4.87 pages visitées, avec un écart type de 3.34.
  
`Statistiques de la variable cible :`
  - **converted** : Taux de conversion moyen de 3.23%.

Le jeu de donnée ne possède aucune donnée manquantes, c'est un point important pour la mise en place de pipeline par la suite. Cependant, il est important de noter que nous sommes dans une situation déséquilibre de classes, aussi appelé imbalanced dataset. Autrement dit, la classe convertie est nettement moins représentée que la convertie.

Il est crucial de prendre en compte ce déséquilibre lors de la modélisation, car les modèles de machine learning peuvent avoir tendance à privilégier la classe majoritaire et à ne pas bien généraliser la classe minoritaire. Cela peut conduire à des prédictions biaisées et peu précises pour la classe sous-représentée.

##
<div style="margin-left: 50px; font-size: 20px;">
0.3 - Explore dataset
</div>

In [ ]:
target = 'converted'

num_features = [c for c in data.columns if c != target]
cat_order = {
    'converted': [0,1]
}

def display_distribution(c):
    fig = px.histogram(data, c, color = target, facet_row = target, histnorm = 'probability')
    fig.update_layout(width=700,height=500)
    fig.update_layout(bargap=0.1)
    fig.show()

In [ ]:
display_distribution(num_features[0])

La majorité des visiteurs sont situés aux US et UK, il est très probable les fondateurs du site soit anglophones. Du moins que le site ne soit pas destiné ou adapté à un public chinois. En effet, les pays européens et US semblent avoir des meilleurs conversions qu'en Chine. Pour quelles raisons ? Une hypothèse serait la barrière de la langue.

In [ ]:
display_distribution(num_features[1])

Comme remarqué sur le descriptif du dataset plus haut, on observe des valeurs d'age extrêmes (>100 ans) et peu representée, sur 280 000 valeurs on peut négliger leur influence et les conserver.

In [ ]:
display_distribution(num_features[3])

La majorité des visiteurs, qu'ils convertissent ou non, arrivent principalement grâce au référencement naturel (SEO). Il est logique que la publicité attire plus de visiteurs sur le site que la recherche directe de l'URL par l'utilisateur. Cependant, le fait que la publicité génère moins de trafic que le référencement naturel suggère que soit la stratégie publicitaire pourrait être améliorée, soit que leur travail de référencement naturel est particulièrement efficace.

In [ ]:
display_distribution(num_features[4])

On devine une tendance d'augmentation de la conversion face au nombre de visite. 

# 1.0 - Preprocessing

##
<div style="margin-left: 50px; font-size: 20px;">
1.1 - Score Log Class
</div>

In [ ]:
class ScoreLog:
    def __init__(self, save_score):
        self.save_score = save_score
        self.load_from_csv()

    def load_from_csv(self):
        if os.path.exists(self.save_score):
            self.df = pd.read_csv(self.save_score)
        else:
            self.df = pd.DataFrame(columns=["len_data", "model_name", "features_list", "f1_score_train", "f1_score_test", "hyperparameters", "datetime","test_size_var", "random_state_var"])

    def log_score(self, len_data, model_name, features_list, f1_score_train, f1_score_test, hyperparameters, test_size_var, random_state_var):
        now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        new_row = {"len_data": len_data, "model_name": model_name, "features_list": features_list, "f1_score_train": f1_score_train, "f1_score_test": f1_score_test, "hyperparameters": hyperparameters, "test_size_var": test_size_var, "random_state_var": random_state_var, "datetime": now}
        self.df = pd.concat([self.df, pd.DataFrame([new_row])], ignore_index=True)
        self.save_to_csv()

    def save_to_csv(self):
        self.df.to_csv(self.save_score, index=False)

    def get_best_score_test(self, model_name=None):
        if model_name:
            filtered_df = self.df[self.df['model_name'] == model_name]
            if filtered_df.empty:
                return None  
            best_score_row = filtered_df.loc[filtered_df['f1_score_test'].idxmax()]
        else:
            best_score_row = self.df.loc[self.df['f1_score_test'].idxmax()]
        return best_score_row

    def get_best_score_train(self, model_name=None):
        if model_name:
            filtered_df = self.df[self.df['model_name'] == model_name]
            if filtered_df.empty:
                return None 
            best_score_row = filtered_df.loc[filtered_df['f1_score_train'].idxmax()]
        else:
            best_score_row = self.df.loc[self.df['f1_score_train'].idxmax()]
        return best_score_row


##
<div style="margin-left: 50px; font-size: 20px;">
1.2 - F1 score class
</div>

In [ ]:
class F1ScoreClassifier:
    def __init__(self, classifier, classifier_name, X_train, X_test, Y_train, Y_test, param_grid={}, cv=3, scoring='f1', verbose=0):
        self.classifier = classifier
        self.classifier_name = classifier_name
        self.X_train = X_train
        self.X_test = X_test
        self.Y_train = Y_train
        self.Y_test = Y_test
        self.param_grid = param_grid
        self.cv = cv
        self.scoring = scoring
        self.verbose = verbose
        self.best_params_ = None
        self.best_score_ = None
        self.best_estimator_ = None
        self.f1_score_train = None
        self.f1_score_test = None
        self.cv_f1_scores = None 
    
    def find_best_params(self):
        gridsearch = GridSearchCV(self.classifier, param_grid=self.param_grid, cv=self.cv, scoring=self.scoring, verbose=self.verbose)
        gridsearch.fit(self.X_train, self.Y_train)
        
        self.best_params_ = gridsearch.best_params_
        self.best_score_ = gridsearch.best_score_
        self.best_estimator_ = gridsearch.best_estimator_
    
    def evaluate_train_test(self):
        Y_train_pred = self.best_estimator_.predict(self.X_train)
        Y_test_pred = self.best_estimator_.predict(self.X_test)
        self.f1_score_train = f1_score(self.Y_train, Y_train_pred)
        self.f1_score_test = f1_score(self.Y_test, Y_test_pred)
        
        print(f"{self.classifier_name} F1-score on train set: {self.f1_score_train}")
        print(f"{self.classifier_name} F1-score on test set: {self.f1_score_test}")
    
    def cross_validate(self):
        cv_scores = cross_val_score(self.best_estimator_, self.X_train, self.Y_train, cv=self.cv, scoring=self.scoring)
        self.cv_f1_scores = cv_scores
        # print(f"{self.classifier_name} Cross-validated F1 scores:", cv_scores)
        print(f"\n{self.classifier_name} Cross-validated F1 score: {cv_scores.mean()}\n")

    def plot_confusion_matrix(self, X, Y, title):
        _, ax = plt.subplots()
        ax.set(title=title)
        ConfusionMatrixDisplay.from_estimator(self.best_estimator_, X, Y, ax=ax)
        plt.show()
    
    def plot_roc_curve(self, X, Y, title): 
        _, ax = plt.subplots()
        ax.set(title=title)
        RocCurveDisplay.from_estimator(self.best_estimator_, X, Y, ax=ax)
        plt.show()

    def order_hyperparameters(self):
        converted_params = {param: float(value) if isinstance(value, (int, float, str)) else value for param, value in self.best_params_.items()}
        self.ordered_hyperparameters = sorted(converted_params.items(), key=lambda x: x[1], reverse=True)

<div style="margin-left: 50px; >

Pour la suite du projet, deux classes ont été créées. La première classe stocke des informations sur chaque entraînement de modèle afin de constituer un historique et d'identifier les hyperparamètres susceptibles d'améliorer le `f1-score`. La seconde classe vise à fluidifier le code, éviter les répétitions et accéder notamment en ce qui concerne l'utilisation de GridSearch (outil permettant de trouver les meilleurs hyperparamètres), accès aux visualisations des résultats des modèles.

Bien qu'il aurait été possible de créer une seule classe pour ces fonctionnalités, j'ai préféré les séparer pour éviter de créer une classe trop généraliste.
</div>

<div style="margin-left: 50px; >
<br>
Rappel


<span style="font-size: 14px">


Le F1-score est calculé à partir des valeurs de True Positive (TP), False Positive (FP) et False Negative (FN).

- <strong>True Positive (TP)</strong> : Le nombre d'échantillons positifs correctement classés.
- <strong>False Positive (FP)</strong> : Le nombre d'échantillons négatifs incorrectement classés comme positifs.
- <strong>False Negative (FN)</strong> : Le nombre d'échantillons positifs incorrectement classés comme négatifs.

La précision (precision) est définie comme le rapport entre TP et la somme de TP et FP :

$$ Precision = \frac{TP}{TP + FP} $$

Le rappel (recall) est défini comme le rapport entre TP et la somme de TP et FN :

$$ Recall = \frac{TP}{TP + FN} $$

Le F1-score est la moyenne harmonique de la précision et du rappel, calculée comme suit :

$$ F1 = 2 \times \frac{Precision \times Recall}{Precision + Recall} $$

Le F1-score prend en compte à la fois la précision et le rappel, ce qui en fait une mesure utile pour évaluer la performance d'un modèle de classification, surtout lorsque les classes sont déséquilibrées.

</span>
</div>

## 
<div style="margin-left: 50px; font-size: 20px;">
1.3 - Preprocessing model
</div>


Nous allons faire une pipeline valable pour tout les modèles de classification en fonction des features initiales. Nous distinguons les variables numériques et catégorielles. Comprenant dans ce premier modèle:
- target: converted
- numerique: age, total_pages_visited
- catégorielle: country, source, new_user

Nous mettons en place un jeu de test représentant 20% des données, avec une pipeline sans features engineering sur les colonnes dans un premier temps.


In [ ]:
categorical_features = ['country', 'source', 'new_user']
numeric_features = ['age', 'total_pages_visited']

features_list = categorical_features+numeric_features

numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(drop="first")

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

X = data[features_list]
Y = data[target]

test_size_var=0.2
random_state_var = 42

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size_var, random_state=random_state_var, stratify=Y)
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

score_logger = ScoreLog('save_score_challenge.csv')

##
<div style="margin-left: 50px; font-size: 20px;">
1.4 - Feature eng LOG , POLY
</div>

Une pipeline avec du features engineering sur les colonnes est potentiellement utilisable. Pour améliorer le code par la suite il sera possible de le rendre plus maléable en offrant la possibilité de choisir la pipeline.

In [ ]:
active = False
if active:
    from sklearn.preprocessing import PolynomialFeatures, FunctionTransformer

    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size_var, random_state=random_state_var, stratify=Y)

    numeric_transformer = StandardScaler()
    categorical_transformer = OneHotEncoder(drop="first")

    poly_transformer = PolynomialFeatures(degree=2)
    log_transformer = FunctionTransformer(np.log1p, validate=True)

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_features),
            ('cat', categorical_transformer, categorical_features),
            ('poly', poly_transformer, numeric_features),
            ('log', log_transformer, numeric_features)
        ]
    )

    pipeline = Pipeline([
        ('preprocessor', preprocessor)
    ])

    X_train = pipeline.fit_transform(X_train)
    X_test = pipeline.transform(X_test)

# 2.0 - Models

##
<div style="margin-left: 50px; font-size: 20px;">
2.1 - First Basic Model
</div>

<div style="margin-left: 50px;">
Dans un premier temps nous allons faire tourner chaque modèle de classification afin d'avoir un premier aperçu des performances de chacun, et de voir vers lesquels s'orienter.</div>

In [ ]:
classifiers = [
    (LogisticRegression(), 'LogisticRegression'),
    (RandomForestClassifier(), 'RandomForestClassifier'),
    (SVC(), 'SVC'),
    (AdaBoostClassifier(),'AdaBoostClassifier'),
    (XGBClassifier(),'XGBRegressorClassifier'),
    (GradientBoostingClassifier(),'GradientBoostingClassifier'),
]
active = False
# active = True

if active:
    for classifier, classifier_name in classifiers:
        cls = F1ScoreClassifier(classifier, classifier_name, X_train, X_test, Y_train, Y_test, param_grid={})
        cls.find_best_params() 
        cls.evaluate_train_test()  
        score_logger.log_score(len_data=len(data), model_name=cls.classifier_name, features_list=features_list, f1_score_train=cls.f1_score_train, f1_score_test=cls.f1_score_test, hyperparameters=cls.best_params_,test_size_var=test_size_var, random_state_var=random_state_var)
        cls.cross_validate()


<div style="margin-left: 50px;">

On peut se rendre compte que la regression logistique, le xgboost et le gradientboost semblent être les modèles les plus aptes à présenter de bons résultats sans overfitter comme pourrait le faire le décision tree.

</div>

<div align="center">

| Model                       | Train                | Test                 | Cross-Val            |
|-----------------------------|----------------------|----------------------|----------------------|
| LogisticRegression         | 0.7617969044922612   | 0.7678300455235205   | 0.761317898096414    |
| GradientBoostingClassifier | 0.7603867653724128   | 0.76381299332119     | 0.7580431606943927   |
| XGBRegressorClassifier     | 0.7711985554134376   | 0.761384335154827    | 0.754029798257489    |
| AdaBoostClassifier         | 0.7484605911330049   | 0.750465549348231    | 0.7486040108067217   |
| SVC                        | 0.7515009746588694   | 0.7499213589178987   | 0.7487032123178571   |
| RandomForestClassifier     | 0.8044642857142857   | 0.7427039904705182   | 0.7312412111296887   |

</div>
<br>
<div style="margin-left: 50px;">
Désormais nous allons étudier chacun des modèles et en tirer les meilleurs résulats possible. Même si un modèle présente de moins résultats en première approche il pourrait être utile pour un futur voting ou stacking.
</div>

##
<div style="margin-left: 50px; font-size: 20px;">
2.2 - Logistic Classification
</div>

In [ ]:
reg_logistic_regression = LogisticRegression()

params_lr = {
    'penalty': ['l1'],
    'C': [26903.536173488137],
    'solver': ['saga'],
    'max_iter':[300],
}


cls_lr = F1ScoreClassifier(reg_logistic_regression, 'LogisticRegression', X_train, X_test, Y_train, Y_test, param_grid=params_lr, cv=5)
cls_lr.find_best_params()
cls_lr.evaluate_train_test()
score_logger.log_score(len_data=len(data), model_name=cls_lr.classifier_name, features_list=features_list, f1_score_train=cls_lr.f1_score_train, f1_score_test=cls_lr.f1_score_test, hyperparameters=cls_lr.best_params_, test_size_var=test_size_var, random_state_var=random_state_var)
cls_lr.cross_validate()

ROC curve matrix

In [ ]:
# preprocessor.get_feature_names_out()
cls_lr.plot_confusion_matrix(cls_lr.X_train, cls_lr.Y_train, "Confusion Matrix on Train set")
cls_lr.plot_confusion_matrix(cls_lr.X_test, cls_lr.Y_test, "Confusion Matrix on Test set")

cls_lr.plot_roc_curve(cls_lr.X_train, cls_lr.Y_train, "ROC Curve on Train set")
cls_lr.plot_roc_curve(cls_lr.X_test, cls_lr.Y_test, "ROC Curve on Test set")

##
<div style="margin-left: 50px; font-size: 20px;">
2.3 - Random Forest
</div>

In [ ]:
reg_random_forest = RandomForestClassifier()

params_rf = {
    'max_depth': [10],
    'min_samples_leaf': [10],
    'min_samples_split': [4],
    'n_estimators': [100],
    'random_state':[42]
}

cls_rf = F1ScoreClassifier(reg_random_forest, 'RandomForestClassifier', X_train, X_test, Y_train, Y_test, param_grid=params_rf, cv=5)
cls_rf.find_best_params()
cls_rf.evaluate_train_test()
score_logger.log_score(len_data=len(data), model_name=cls_rf.classifier_name, features_list=features_list, f1_score_train=cls_rf.f1_score_train, f1_score_test=cls_rf.f1_score_test, hyperparameters=cls_rf.best_params_, test_size_var=test_size_var, random_state_var=random_state_var)
cls_rf.cross_validate()

In [ ]:
cls_rf.plot_confusion_matrix(cls_rf.X_train, cls_rf.Y_train, "Confusion Matrix on Train set")
cls_rf.plot_confusion_matrix(cls_rf.X_test, cls_rf.Y_test, "Confusion Matrix on Test set")

cls_rf.plot_roc_curve(cls_rf.X_train, cls_rf.Y_train, "ROC Curve on Train set")
cls_rf.plot_roc_curve(cls_rf.X_test, cls_rf.Y_test, "ROC Curve on Test set")

##
<div style="margin-left: 50px; font-size: 20px;">
2.4 - SVM
</div>

In [ ]:
reg_svc = SVC()

# params_svc = {
#     'C': [0.1, 1],
#     'kernel': ['rbf'],
#     'gamma': ['scale', 'auto']
# }

params_svc = {
    'C': [3],
}

cls_svc = F1ScoreClassifier(reg_svc, 'SVC', X_train, X_test, Y_train, Y_test, param_grid=params_svc, cv=5)
cls_svc.find_best_params()
cls_svc.evaluate_train_test()

score_logger.log_score(len_data=len(data), model_name=cls_svc.classifier_name, features_list=features_list, f1_score_train=cls_svc.f1_score_train, f1_score_test=cls_svc.f1_score_test, hyperparameters=cls_svc.best_params_,test_size_var=test_size_var, random_state_var=random_state_var)
cls_svc.cross_validate()

##
<div style="margin-left: 50px; font-size: 20px;">
2.5 - XGBoost
</div>

In [ ]:
xgboost = XGBClassifier()


params = {
    'booster': ['gblinear'],
    'objective': ['binary:logistic'],
    'learning_rate': [0.3],
    'eta': [0.2],
    'n_estimators': [800],
    'scale_pos_weight': [1.84289]
}

# params = {}

cls_xgb = F1ScoreClassifier(xgboost, 'XGBClassifier', X_train, X_test, Y_train, Y_test, param_grid=params,scoring = 'f1', cv=5)
cls_xgb.find_best_params()
cls_xgb.evaluate_train_test()

score_logger.log_score(len_data=len(data), model_name=cls_xgb.classifier_name, features_list=features_list, f1_score_train=cls_xgb.f1_score_train, f1_score_test=cls_xgb.f1_score_test, hyperparameters=cls_xgb.best_params_,test_size_var=test_size_var, random_state_var=random_state_var)
cls_xgb.cross_validate()


In [ ]:
cls_xgb.plot_confusion_matrix(cls_xgb.X_train, cls_xgb.Y_train, "Confusion Matrix on Train set")
cls_xgb.plot_confusion_matrix(cls_xgb.X_test, cls_xgb.Y_test, "Confusion Matrix on Test set")

cls_xgb.plot_roc_curve(cls_xgb.X_train, cls_xgb.Y_train, "ROC Curve on Train set")
cls_xgb.plot_roc_curve(cls_xgb.X_test, cls_xgb.Y_test, "ROC Curve on Test set")

##
<div style="margin-left: 50px; font-size: 20px;">
2.6 - AdaBoost
</div>

In [ ]:
decision_tree = DecisionTreeClassifier()
adaboost_dt = AdaBoostClassifier(estimator=decision_tree)

params = {
    'estimator__max_depth': [4],
    'estimator__min_samples_leaf': [2],
    'estimator__min_samples_split': [4],
    'n_estimators': [8]
}
# params={}
cls_ada = F1ScoreClassifier(adaboost_dt, 'AdaBoostClassifier', X_train, X_test, Y_train, Y_test, param_grid=params, cv=5)
cls_ada.find_best_params()
cls_ada.evaluate_train_test()

score_logger.log_score(len_data=len(data), model_name=cls_ada.classifier_name, features_list=features_list, f1_score_train=cls_ada.f1_score_train, f1_score_test=cls_ada.f1_score_test, hyperparameters=cls_ada.best_params_,test_size_var=test_size_var, random_state_var=random_state_var)
cls_ada.cross_validate()

##
<div style="margin-left: 50px; font-size: 20px;">
2.7 - GradientBoosting
</div>

In [ ]:
gradientboost = GradientBoostingClassifier()

params = {
    'max_depth': [8],
    'min_samples_leaf': [10],
    'min_samples_split': [8],
    'n_estimators': [48]
}

cls_gb = F1ScoreClassifier(gradientboost, 'GradientBoostingClassifier', X_train, X_test, Y_train, Y_test, param_grid=params, cv=5)
cls_gb.find_best_params()
cls_gb.evaluate_train_test()

score_logger.log_score(len_data=len(data), model_name=cls_gb.classifier_name, features_list=features_list, f1_score_train=cls_gb.f1_score_train, f1_score_test=cls_gb.f1_score_test, hyperparameters=cls_gb.best_params_,test_size_var=test_size_var, random_state_var=random_state_var)
cls_gb.cross_validate()

In [ ]:
cls_xgb.plot_confusion_matrix(cls_xgb.X_train, cls_xgb.Y_train, "Confusion Matrix on Train set")
cls_xgb.plot_confusion_matrix(cls_xgb.X_test, cls_xgb.Y_test, "Confusion Matrix on Test set")

cls_xgb.plot_roc_curve(cls_xgb.X_train, cls_xgb.Y_train, "ROC Curve on Train set")
cls_xgb.plot_roc_curve(cls_xgb.X_test, cls_xgb.Y_test, "ROC Curve on Test set")

# 3.0 - Voting & Stacking

##
<div style="margin-left: 50px; font-size: 20px;">
3.1 - Voting
</div>

In [ ]:
cls_lr_best = LogisticRegression(penalty='l1', C=26903.536173488137, solver='saga', max_iter=300)
cls_xgb_best = XGBClassifier(booster='gblinear', objective = 'binary:logistic',learning_rate = 0.3,eta = 0.2,n_estimators=800,scale_pos_weight =1.84289)
cls_random_forest_best = RandomForestClassifier(max_depth=10, min_samples_leaf=10, min_samples_split=4, n_estimators=100)
cls_gradientboost_best = GradientBoostingClassifier(max_depth = 8,min_samples_leaf=10,min_samples_split = 8,n_estimators = 48)
cls_adaboost_dt_best = AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=4, min_samples_leaf=2, min_samples_split=4),n_estimators=8)
cls_svc_best = SVC()


voting = VotingClassifier(
    estimators=[("logistic", cls_lr_best), ("xgboost_best", cls_xgb_best)],
    # voting="hard",
    voting="soft",
)

voting.fit(X_train, Y_train)
y_pred_train = voting.predict(X_train)
y_pred_test = voting.predict(X_test)

f1_train = f1_score(Y_train, y_pred_train)
f1_test = f1_score(Y_test, y_pred_test)

print("F1 score on training set:", f1_train)
print("F1 score on test set:", f1_test)

cv_scores = np.mean(cross_val_score(voting, X_train, Y_train, cv=5, scoring='f1'))
print(np.mean(cv_scores))

##
<div style="margin-left: 50px; font-size: 20px;">
3.2 - Stacking
</div>

In [ ]:
active = False
active = True

if active:
    cls_lr_best = LogisticRegression(penalty='l1', C=26903.536173488137, solver='saga', max_iter=300)
    cls_xgb_best = XGBClassifier(booster='gblinear', objective = 'binary:logistic',learning_rate = 0.3,eta = 0.2,n_estimators=800,scale_pos_weight =1.84289)

    stacking = StackingClassifier(
        estimators=[("reg_logistic_regression_best", cls_lr_best), ("xgboost_best", cls_xgb_best)],
    )

    preds = stacking.fit_transform(X_train, Y_train)
    predictions = pd.DataFrame(preds, columns=stacking.named_estimators_.keys())


    stacking.fit(X_train, Y_train)

    y_pred_train = stacking.predict(X_train)
    y_pred_test = stacking.predict(X_test)

    f1_train = f1_score(Y_train, y_pred_train)
    f1_test = f1_score(Y_test, y_pred_test)

    print("F1 score on training set:", f1_train)
    print("F1 score on test set:", f1_test)

    cv_scores = np.mean(cross_val_score(stacking, X_train, Y_train, cv=3, scoring='f1'))
    print(np.mean(cv_scores))

    corr_matrix = predictions.corr().round(2)
    import plotly.figure_factory as ff

    fig = ff.create_annotated_heatmap(corr_matrix.values, x=corr_matrix.columns.tolist(), y=corr_matrix.index.tolist())
    fig.show()

##
<div style="margin-left: 50px; font-size: 20px;">
3.3 - Voting+Stacking (Frankenstein)
</div>

In [ ]:
cls_lr_best = LogisticRegression(penalty='l1', C=26903.536173488137, solver='saga', max_iter=300)
cls_xgb_best = XGBClassifier(booster='gblinear', objective = 'binary:logistic',learning_rate = 0.3,eta = 0.2,n_estimators=800,scale_pos_weight =1.84289)
cls_random_forest_best = RandomForestClassifier(max_depth=10, min_samples_leaf=10, min_samples_split=4, n_estimators=100)
cls_gradientboost_best = GradientBoostingClassifier(max_depth = 8,min_samples_leaf=10,min_samples_split = 8,n_estimators = 48)
cls_adaboost_dt_best = AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=4, min_samples_leaf=2, min_samples_split=4),n_estimators=8)
cls_svc_best = SVC(probability=True)


voting1 = VotingClassifier(
    estimators=[("cls_adaboost_dt_best", cls_adaboost_dt_best), ("cls_svc_best", cls_svc_best)],   
    voting="soft",
)

voting2 = VotingClassifier(
    estimators=[("cls_lr_best", cls_lr_best), ("cls_xgb_best", cls_xgb_best)], 
    voting="soft",
)

stacking = StackingClassifier(
    estimators=[("adaboost_svc", voting1), ("lr_xgb", voting2)],
)


preds = stacking.fit_transform(X_train, Y_train)
predictions = pd.DataFrame(preds, columns=stacking.named_estimators_.keys())

stacking.fit(X_train, Y_train)

y_pred_train = stacking.predict(X_train)
y_pred_test = stacking.predict(X_test)

f1_train = f1_score(Y_train, y_pred_train)
f1_test = f1_score(Y_test, y_pred_test)

print("F1 score on training set:", f1_train)
print("F1 score on test set:", f1_test)

cv_scores = np.mean(cross_val_score(stacking, X_train, Y_train, cv=3, scoring='f1'))
print('cv_score',np.mean(cv_scores))

corr_matrix = predictions.corr().round(2)
import plotly.figure_factory as ff

fig = ff.create_annotated_heatmap(corr_matrix.values, x=corr_matrix.columns.tolist(), y=corr_matrix.index.tolist())
fig.update_layout(width=700)
fig.show()

##


<div align="center">

| Model                                | Train                | Test                 | Cross-Val            |
|--------------------------------------|----------------------|----------------------|----------------------|
| XGBRegressorClassifier_gs           | 0.7698390677025527   | 0.7722330638416504   | 0.7688006973234403   |
| stacking_xgb_lr                     | 0.7689066472938613   | 0.7735354124162052   | 0.7680804298273997   |
| voting_xgb_lr                       | 0.7685848715164676   | 0.7750724637681159   | 0.7678745168257773   |
| LogisticRegression_gs               | 0.7618832050701675   | 0.7684848484848484   | 0.7620626784328032   |
| RandomForestClassifier_gs          | 0.7697651589518991   | 0.7595013681970204   | 0.7567297334531815   |
| AdaBoostClassifier_gs               | 0.7632772494513533   | 0.7660818713450293   | 0.7570407079224383   |
| GradientBoostingClassifier_gs        | 0.7788937190825963   | 0.760928549894483    | 0.7511303484163259   |

</div>

# 4.0 - Best Score

Section to look at the best results on test or train f1-score for all models or per model.

##
<div style="margin-left: 50px; font-size: 20px;">
4.1 - By Model
</div>

In [ ]:
name = {
    0: 'LogisticRegression',
    1: 'RandomForestClassifier',
    2: 'SVC',
    3: 'AdaBoostClassifier',
    4: 'XGBRegressorClassifier',
    5: 'GradientBoostingClassifier'
}
name_nb=3
best_score_by_model = score_logger.get_best_score_test(model_name=name_nb)
print(f"Best score for {name.get(name_nb)} model:", best_score_by_model)



##
<div style="margin-left: 50px; font-size: 20px;">
4.2 - Over all Models
</div>

In [ ]:
f1_score_best = score_logger.get_best_score_test()
print("Best Score:")
print(f1_score_best)


# 5.0 - SAVE MODEL

Section destinate to save a model

##
<div style="margin-left: 50px; font-size: 20px;">
5.1 - Save a Model
</div>

In [ ]:
cls_xgb_best = XGBClassifier(booster='gblinear', objective = 'binary:logistic',learning_rate = 0.3,eta = 0.2,n_estimators=800,scale_pos_weight =1.84289)

model_name = 'cls_xgb_best'
classifier = cls_xgb_best

active = False
# active = True
if active:
    X = np.append(X_train,X_test,axis=0)
    Y = np.append(Y_train,Y_test)

    classifier.fit(X,Y)
    data_without_labels = pd.read_csv('conversion_data_test.csv')
    X_without_labels = data_without_labels.loc[:, features_list]

    # X_without_labels = X_without_labels.values
    X_without_labels = preprocessor.transform(X_without_labels)

    data_pred = {
        'converted': classifier.predict(X_without_labels)
    }

    Y_predictions = pd.DataFrame(columns=['converted'],data=data_pred)
    csv_name = f"conversion_data_test_predictions_AntoineV-model_{model_name}.csv"
    Y_predictions.to_csv(csv_name, index=False)

##
<div style="margin-left: 50px; font-size: 20px;">
5.2 - Save "Best" Model
</div>

In [ ]:
model_name = f1_score_best['model_name']
hyperparameters = f1_score_best['hyperparameters']

classifier = eval(model_name)(**hyperparameters)

active = False
# active = True

if active:
    X = np.append(X_train,X_test,axis=0)
    Y = np.append(Y_train,Y_test)

    classifier.fit(X,Y)
    data_without_labels = pd.read_csv('conversion_data_test.csv')
    X_without_labels = data_without_labels.loc[:, features_list]

    # X_without_labels = X_without_labels.values
    X_without_labels = preprocessor.transform(X_without_labels)

    data_pred = {
        'converted': classifier.predict(X_without_labels)
    }

    Y_predictions = pd.DataFrame(columns=['converted'],data=data_pred)
    csv_name = f"conversion_data_test_predictions_AntoineV-model_{model_name}_{str(f1_score_best['f1_score_test'])}.csv"
    Y_predictions.to_csv(csv_name, index=False)
    print('new best classifier',model_name,f1_score_best)


# 6 - Conclusion

Pour résumer ce projet, nous avons entrepris une analyse approfondie des `données déséquilibrées` (imbalanced data), ce qui a nécessité une démarche méthodique pour garantir des résultats fiables et significatifs. Notre objectif principal était de développer des `modèles de machine learning` capables de prédire efficacement une variable cible mal équilibrée.

Dans notre démarche, nous avons tout d'abord identifié le déséquilibre des classes dans nos données et ses implications sur la performance des modèles. Ensuite, nous avons sélectionné plusieurs approches pour traiter ce déséquilibre, notamment l'utilisation d'une métrique d'évaluation adaptée: le F1-score.

En ce qui concerne les résultats, plusieurs modèles ont démontré des performances prometteuses. Notamment, le modèle `XGBoost` et la `régression logistique` ont affiché des performances solides. De plus, lorsque nous les avons combinés dans un cadre de voting ou de stacking, nous avons observé une légère amélioration des performances (`voting_xgb_lr` et `stacking_xgb_lr`). Ces combinaisons de modèles ont permis de capitaliser sur les forces individuelles de chaque algorithme, conduisant ainsi à des performances globales plus élevées.


En conclusion, ce projet met en lumière l'importance de prendre en compte le déséquilibre des classes lors de la modélisation des données. Les approches et les techniques que nous avons explorées ont permis d'améliorer la performance des modèles et de produire des prédictions plus précises. Cependant, il reste encore des possibilités d'amélioration, notamment en explorant davantage de techniques spécifiques aux données déséquilibrées et en affinant les paramètres des modèles pour obtenir des performances encore meilleures. Des librairies comme `imbalanced-learn` existent et permettent de prendre en charge ces cas d'imbalanced data.

Un constat important est le `faible taux de conversion` des acheteurs `chinois`, bien qu'ils représentent un groupe démographique significatif. La priorité principale est donc de conseiller à l'équipe produit de réviser la version chinoise du site pour garantir un `contenu adapté`: traductions précises, options de paiement appropriées.

Etant donné que le site réussit à convertir les acheteurs de moins de `40 ans`, l'équipe marketing devrait concentrer ses efforts sur ce groupe à travers des publicités. Il serait bénéfique de représenter le site (retargeting) aux visiteurs qui ont consulté de `nombreuses pages` mais n'ont pas encore franchi le pas, car c'est un indicateur positif de `conversion potentielle`.